# Final Project

## Imports

In [17]:
# <include-final_project/utils.py>

In [18]:
# <imports>
import numpy as np
import pandas as pd
import plotly.io as pio


from final_project import utils

pd.options.plotting.backend = "plotly"
pio.templates.default = "seaborn"
pio.renderers.default = "notebook_connected"

### Markets

In [19]:
df_exch = utils.get_exchange_info()
df_exch.loc["BTCUSDT"]

status                                                                  TRADING
baseAsset                                                                   BTC
baseAssetPrecision                                                            8
quoteAsset                                                                 USDT
quotePrecision                                                                8
quoteAssetPrecision                                                           8
baseCommissionPrecision                                                       8
quoteCommissionPrecision                                                      8
orderTypes                    [LIMIT, LIMIT_MAKER, MARKET, STOP_LOSS_LIMIT, ...
icebergAllowed                                                             True
ocoAllowed                                                                 True
quoteOrderQtyMarketAllowed                                                 True
isSpotTradingAllowed                    

## Contracts

In [26]:
df_perpetual = utils.get_continuous_contracts(pair="BTCUSDT", start_time="2021-05-01")
fig = utils.make_price_volume_chart(df_perpetual, title="BTCUSDT Perpetual Contracts 8 Hour Intervals")
fig.show()

In [21]:
df_perpetual = utils.get_continuous_contracts(pair="BTCUSDT", start_time="2021-05-01", contract_type="CURRENT_QUARTER")
fig = utils.make_price_volume_chart(df_perpetual, title="BTCUSDT Current Quarter Contracts 8 Hour Intervals")
fig.show()

## Spot Prices

These are the same prices as above.

In [27]:
df_spot = utils.get_klines(symbol="BTCUSDT", start_time="2021-05-01")
fig = utils.make_price_volume_chart(df_spot, title="BTCUSDT OHLC 8 Spot Price Hour Intervals")
fig.show()

## Funding Rate

In [23]:
df_funding = utils.get_funding_rate_history(symbol="BTCUSDT", start_time="2021-04-30 16:00")
fig = df_funding.fundingRate.plot(title="BTCUSDT Funding Rate")
fig.update_layout(showlegend=False)
fig.show()

In [24]:
fig = utils.make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(utils.go.Scatter(x=df_perpetual.index, y= np.log(df_perpetual.close / df_spot.close), name="spread"), secondary_y=False)
fig.add_trace(utils.go.Scatter(x=df_funding.index, y=df_funding.fundingRate, name="funding_rate"), secondary_y=True)
fig.update_layout(title="Perpetual Price as a % of Spot vs. Funding Rate")
fig.show()